In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [4]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Combined.csv'
df = pd.read_csv(dataset_path)

<ipython-input-4-970b3fbba648>:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dataset_path)


In [5]:
null_col = [key for key in dict(df.isnull().sum().sort_values(ascending=False))
              if dict(df.isnull().sum().sort_values(ascending=False))[key] > 937218]
print(null_col)

['dVid', 'sVid', 'DstWin', 'DstTCPBase', 'SrcWin', 'dDSb', 'dHops', 'dTtl', 'dTos', 'SrcTCPBase', 'DstGap', 'SrcGap']


In [6]:
df.isnull().sum().sort_values(ascending=False)

dVid           1213881
sVid           1101319
DstWin         1038812
DstTCPBase      985843
SrcWin          973470
dDSb            943067
dHops           943067
dTtl            943067
dTos            943067
SrcTCPBase      937219
DstGap          937219
SrcGap          937219
sTos               214
sDSb               214
sTtl               214
sHops              214
DstRate              0
Rate                 0
SrcRate              0
pLoss                0
DstLoss              0
Unnamed: 0           0
State                0
Loss                 0
TcpRtt               0
SynAck               0
AckDat               0
Label                0
Attack Type          0
SrcLoss              0
dMeanPktSz           0
DstLoad              0
TotPkts              0
Dur                  0
RunTime              0
Mean                 0
Sum                  0
Min                  0
Max                  0
Proto                0
Cause                0
SrcPkts              0
SrcLoad              0
DstPkts    

In [7]:
#dropping column with greater null values
df.drop(columns = null_col,inplace = True)

In [8]:
#remaing null columns with less number of null values to be filled with zero
null_col_2 = [key for key in dict(df.isnull().sum().sort_values(ascending=False))
              if dict(df.isnull().sum().sort_values(ascending=False))[key] > 0]
print(null_col_2)

['sTtl', 'sHops', 'sTos', 'sDSb']


In [9]:
df.loc[:, null_col_2] = df.loc[:, null_col_2].fillna(0)
df.drop(columns = 'Unnamed: 0',inplace = True)

In [10]:
#Performing Under sampling
df['Label'].value_counts()

Label
Malicious    738153
Benign       477737
Name: count, dtype: int64

In [11]:
df_be = df[df['Label'] == 'Benign']
df_ma = df[df['Label'] == 'Malicious']

In [12]:
#selecting 5000 rows for each class
df_be_5000 = df_be.iloc[:5000]
df_ma_5000 = df_ma.iloc[:5000]
df_10000 = pd.concat([df_be_5000,df_ma_5000],ignore_index = True)
df_10000 = df_10000.sample(frac =1).reset_index(drop = True)
df_10000['Label'].value_counts()

Label
Benign       5000
Malicious    5000
Name: count, dtype: int64

In [13]:
#seprating target variable from the data set
X = df_10000.copy()
X.drop(columns = 'Label',inplace = True)
y = df_10000[['Label']]

In [14]:
X.drop(columns = 'Attack Type',inplace = True)
X.drop (columns = 'Attack Tool',inplace = True)

In [15]:
#Categorical variables
cat = [key for key in dict(X.dtypes)
        if dict(X.dtypes)[key] in ['object']]
print(cat)

['Proto', 'sDSb', 'Cause', 'State']


In [16]:
X[cat] = X[cat].astype(str)
ohe = OneHotEncoder(handle_unknown='ignore')
Xcat = pd.DataFrame(ohe.fit_transform(X[cat]).toarray(),columns=ohe.get_feature_names_out(),index=X.index)
X = pd.concat([X,Xcat],axis=1)
X.drop(labels=cat,axis=1,inplace=True)
X.sample(5)


,Seq,Dur,RunTime,Mean,Sum,Min,Max,sTos,sTtl,sHops,...,sDSb_ef,Cause_Start,Cause_Status,State_CON,State_ECO,State_FIN,State_INT,State_REQ,State_RST,State_TST
7174,4371,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,49.0,15.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
631,4789,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,44.0,20.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5627,134,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,57.0,7.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2612,6984,0.212215,0.212215,0.212215,0.212215,0.212215,0.212215,0.0,58.0,6.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2835,6333,2.996714,2.996714,2.996714,2.996714,2.996714,2.996714,136.0,255.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
#Label encoder
label  = LabelEncoder()
y = label.fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
#Implementing XGBoost
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X, y)
y_pred = xgb_model.predict(X_test)

In [20]:
target_names = ['Benign','Malicious']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      Benign       1.00      1.00      1.00      1658
   Malicious       1.00      1.00      1.00      1642

    accuracy                           1.00      3300
   macro avg       1.00      1.00      1.00      3300
weighted avg       1.00      1.00      1.00      3300

